# Парсинг информации о странах и их рейтинга

## Подключение нужных библиотек

In [1]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from tqdm import tqdm

##  Парсинг стран и их рейтинга в Happiness Score

In [2]:
def get_happiness_data(url = "https://en.wikipedia.org/wiki/World_Happiness_Report"):
    """
    Собирает информацию о странах и их рейтинге в Happiness Score с Википедии
    Возвращает: DataFrame
    """

    try:
        happiness_df = pd.read_html(url)[15]
    except Exception as e:
        print(f"Error reading HTML tables: {e}")
        return pd.DataFrame()

    happiness_df.rename(columns={'Overall rank': 'Happiness rank',
                                 'Country or region': 'Country',
                                 'Life evaluation': 'Happiness score'},
                        inplace=True)
    return happiness_df

In [3]:
countries_happiness_df = get_happiness_data()

os.makedirs('data', exist_ok=True)
countries_happiness_df.to_csv('data/countries_happiness.csv', index=False)

## Парсинг различных показателей стран с World Bank Open Data

In [4]:
BASE_URL = "https://data.worldbank.org"

def get_country_links(base_url=BASE_URL):
    """
    Достает ссылки на все страны из подкаталога country
    Возвращает массив пар: (Название страны, Ссылка на страну)
    """
    url = urljoin(base_url, "country")
    print(f"Ищем ссылки на страны на странице: {url}")
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, "html.parser")
    country_links = []

    # Find all <a> tags whose href begins with "/country/"
    for a in soup.find_all("a", href=True):
        href = a['href']
        if href.startswith("/country/"):
            country_name = a.get_text(strip=True)
            full_url = urljoin(base_url, href)
            # Avoid duplicates
            if country_name and full_url not in [link for (_, link) in country_links]:
                country_links.append((country_name, full_url))

    print(f"Найдено {len(country_links)} стран.")
    return country_links

In [8]:
def get_country_indicators(country_url):
    """
    Принимает URL страницы страны (например, "https://data.worldbank.org/country/finland?view=chart")
    и возвращает:
      - словарь с данными индикаторов: {название индикатора: значение (float или None)},
      - словарь с метаданными: {название индикатора: ссылка на индикатор}.
    """

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(country_url)

    wait = WebDriverWait(driver, 5)
    try:
        # Ищем вкладку "By Theme" по XPath (элемент <li> с классом react-tabs__tab, внутри которого есть <span> с
        # нужным текстом)
        by_theme_tab = wait.until(
            ec.element_to_be_clickable(
                (By.XPATH, "//li[contains(@class, 'react-tabs__tab') and .//span[contains(text(), 'By Theme')]]")
            )
        )
        # Кликаем на нужную вкладку
        driver.execute_script("arguments[0].click();", by_theme_tab)

        # Ждем появления контента вкладки
        wait.until(ec.presence_of_element_located((By.CSS_SELECTOR, ".indicator-item")))
        time.sleep(2)  # небольшая задержка
        html = driver.page_source
        # print(f"Вкладка 'By Theme' успешно открыта для {country_url}")
    except Exception as e:
        print(f"Не удалось найти или кликнуть по вкладке 'By Theme' для {country_url}: {e}")
        driver.quit()
        return {}, {}
    finally:
        driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    indicators_info = soup.find_all(class_='indicator-item__inner')

    indicator_values = {}
    indicator_links = {}

    for indicator_info in indicators_info:
        headline_elem = indicator_info.find(class_='indicator-item__headline-mobile')
        if headline_elem:
            a_tag = headline_elem.find('a')
            if a_tag:
                indicator_name = a_tag.get_text(strip=True)
                indicator_link = a_tag.get('href')
                indicator_link = indicator_link.split('?locations=')[0]
                indicator_links[indicator_name] = indicator_link

                data_info = indicator_info.find(class_='indicator-item__data-info')
                if data_info:
                    span_tag = data_info.find('span')
                    if span_tag:
                        try:
                            # Убираем запятые, пробуем преобразовать текст в число
                            value = float(span_tag.get_text(strip=True).replace(',', ''))
                        except Exception as ex:
                            print(f"Ошибка преобразования значения для '{indicator_name}': {ex}")
                            value = None
                    else:
                        value = None
                else:
                    value = None
                indicator_values[indicator_name] = value

    return indicator_values, indicator_links

### Собираем информацию по всем индикаторам для всех стран

In [11]:
countries = get_country_links()[:5]

all_country_data = []  # список словарей: каждый словарь содержит данные для одной страны
metadata_dict = {}  # метаданные (предполагается, что структура индикаторов одинакова для всех стран)

for country_name, country_url in (pbar := tqdm(countries)):
    pbar.set_description(f"Собираем информацию о стране {country_name}")
    indicators, metadata = get_country_indicators(country_url)

    metadata = {indicator_name: urljoin(BASE_URL, metadata[indicator_name]) for indicator_name in metadata.keys()}

    # Формируем словарь с данными: добавляем имя страны и пары название индикатора – значение
    data_entry = {"Country": country_name}
    data_entry.update(indicators)
    all_country_data.append(data_entry)
    # Обновляем метаданные (если индикаторы совпадают, то они будут перезаписаны, что нормально)
    metadata_dict.update(metadata)
    time.sleep(2)  # пауза между запросами к разным странам


Ищем ссылки на страны на странице: https://data.worldbank.org/country
Найдено 217 стран.


Собираем информацию о стране Andorra: 100%|██████████| 5/5 [01:39<00:00, 19.91s/it]       


### Объединяем всю информацию в один DataFrame и сохраняем

In [10]:
os.makedirs('data', exist_ok=True)

# Создаем DataFrame, где индекс – название страны, а столбцы – названия индикаторов
df = pd.DataFrame(all_country_data)
df.set_index("Country", inplace=True)
df.to_csv("data/countries_indicators.csv", encoding="utf-8")
print("Сохранены данные по индикаторам для стран в файле 'data/country_indicators.csv'.")

# Сохраняем метаданные в отдельный CSV: для каждого индикатора его ссылка
meta_df = pd.DataFrame(list(metadata_dict.items()), columns=["Indicator", "Link"])
meta_df.to_csv("data/indicators_metadata.csv", index=False, encoding="utf-8")
print("Сохранены метаданные индикаторов в файле 'data/indicators_metadata.csv'.")


Сохранены данные по индикаторам для стран в файле 'country_indicators.csv'.
Сохранены метаданные индикаторов в файле 'indicators_metadata.csv'.
